In [1]:
%pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

import random

## Carga de datos

In [4]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [5]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [6]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [7]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [8]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [9]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [10]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [11]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [12]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [13]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [14]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [15]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [16]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [17]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [18]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [19]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [20]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [21]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [22]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.




In [49]:
len(newsgroups_train.data)
idxs = [random.randint(0, len(newsgroups_train.data) - 1) for i in range(5)]
print(idxs)
docs = [newsgroups_train.data[idx] for idx in idxs]
Xs_train = X_train[idxs]
Ys_train = y_train[idxs]

for i in range(len(Ys_train)):
    print(newsgroups_train.target_names[Ys_train[i]])

[7143, 1274, 905, 10219, 10376]
sci.space
comp.windows.x
soc.religion.christian
sci.space
talk.religion.misc


In [51]:
# print(idxs)
# Medimos similitud con similitud coseno
cossim = cosine_similarity(Xs_train, X_train)

target_list = list()
result_idxs = np.zeros(shape=(5,5), dtype=int)
for i, element in enumerate(cossim):
    print("\n\nDocumento: ", i+1, "IDX: ", idxs[i], " Target: ", newsgroups_train.target_names[Ys_train[i]])
    sort_result = np.argsort(element)
    sort_result = sort_result[sort_result != idxs[i]]
    result_idxs[i] = sort_result[::-1][1:6]

    target_list = list()
    print("Target primeros 5 documentos con mayor similitud:")
    for sort_idx in result_idxs[i]:
        target_list.append(newsgroups_train.target_names[y_train[sort_idx]])
    print(target_list)
    print("IDX primeros 5 documentos con mayor similitud:")
    print(result_idxs[i])

    print(element[result_idxs[i]])

    # print(element)



Documento:  1 IDX:  7143  Target:  sci.space
Target primeros 5 documentos con mayor similitud:
['soc.religion.christian', 'sci.space', 'talk.politics.mideast', 'soc.religion.christian', 'sci.space']
IDX primeros 5 documentos con mayor similitud:
[5572 5052 2527 7585 9227]
[0.17219445 0.16617523 0.16358265 0.16036211 0.15561626]


Documento:  2 IDX:  1274  Target:  comp.windows.x
Target primeros 5 documentos con mayor similitud:
['comp.windows.x', 'comp.windows.x', 'comp.windows.x', 'comp.windows.x', 'comp.windows.x']
IDX primeros 5 documentos con mayor similitud:
[ 367 6657 9733 7561 7967]
[0.39695861 0.38896964 0.38359257 0.37404395 0.3636564 ]


Documento:  3 IDX:  905  Target:  soc.religion.christian
Target primeros 5 documentos con mayor similitud:
['soc.religion.christian', 'soc.religion.christian', 'talk.religion.misc', 'soc.religion.christian', 'soc.religion.christian']
IDX primeros 5 documentos con mayor similitud:
[ 7238    51 11196  8868  2043]
[0.18019744 0.16390929 0.1632

    Respecto a si el resultado obtenido tiene sentido la similitud según la etiqueta de clasificación tenemos:
- Documento 1:

    El documento objetivo está clasificado como sci.space. Vemos que solo dos de los 5 más cercanos tiene la misma categoría (y no son los de mayor similitud).

- Documento 2:

    El documento objetivo está clasificado como com.windows.x. En este caso los primeros 5 textos con mayor similitud tienen la misma categoría.

- Documento 3: 

    El documento objetivo está clasificado como soc.religion.christian. 4 de los 5 documentos están categorizados de la misma manera, el que difiere si bien es de otra categoría, por el nombre de la misma talk.religion.misc podríamos suponer que se acerca bastante a la categoría objetiva.

- Documento 4:
    
    El documento objetivo está clasificado como sci.space. En este caso los 5 documentos con mayor similitud también son de la misma categoría.

- Documento 5: 

    El ducomento objetivo está clasificado como talk.religion.misc. 3 de los 5 documentos tienen la misma clasificación. Los 2 restante que son de talk.politics.guns. Estos son el tercero y cuarto de mayor similitud.  
  

In [15]:
result_idxs

array([[ 3404,  1459,   585,  6075,  7972],
       [ 3591,  4230,  1693,  2829,  8277],
       [10825,  3259,  3864,  9347,  1030],
       [ 1390,  2838,  9759,  3166,  9623],
       [ 9635,  9252, 10480,  4113,   683]])

In [65]:
# Documento 1
index_doc = 0

print("DOCUMENTO: ", index_doc+1, "  - TIPO: ", newsgroups_train.target_names[Ys_train[index_doc]])
print(newsgroups_train.data[idxs[index_doc]])
print("\n----------------------------------------------------------------\n")
for i in range(5):
    print("")
    print(newsgroups_train.data[result_idxs[index_doc,i]])
    print("\n\n/////////////////////////////\n\n")

DOCUMENTO:  1   - TIPO:  sci.space





Didn't one of the early jet fighters have these?
I also think the germans did some work on these in WWII.


A lot of this was also done by the military...


Weren't the first microwave landing systems from WWII too?


Egad! I'm disagreeing with Mary Shafer!

----------------------------------------------------------------

A few points about Mary's being taken into heaven at the end of her life on
earth:

One piece of evidence for Mary's assumption into heaven is the fact
that no Christian church ever claimed to be the sight where she was
buried. Some Christian churches claimed to be located at the final
resting places of Peter, Mark, and other saints, but no one ever
claimed to possess the body of Mary, the greatest of the saints. Why?
Because everyone knew that she had been taken up into heaven.

Although there is no definitive scriptural proof for the assumption of
Mary, some passages seem suggestive, like the passage in Revelation
that descri

Respecto al contenido de los textos, vemos que no tienen similitud entre ellos. Los que si, observamos algunas palabras que se repiten entre ellos, que entendemos que son las responsables de que el de la mayor similitud del coseno. Entre ellas podemos listar: Mary, Nasa, WWII, German

In [66]:
# Documento 2
index_doc = 1

print("DOCUMENTO: ", index_doc+1, "  - TIPO: ", newsgroups_train.target_names[Ys_train[index_doc]])
print(newsgroups_train.data[idxs[index_doc]])
print("\n----------------------------------------------------------------\n")
for i in range(5):
    print(newsgroups_train.data[result_idxs[index_doc,i]])
    print("\n\n/////////////////////////////\n\n")

DOCUMENTO:  2   - TIPO:  comp.windows.x
Hello Motif World,

a few days ago I posted my announcement for an update of Motif++. I got
several requests to send the bindings per e-mail, and I know of several people
who have been using Motif++, and there are probably a number of people I am
not aware of who are also using Motif++.

My question is:

How many people 'out there' would be interested to join a mailing-list, where
people can ask questions about Motif++, swap stories, and give new ideas about
new directions and improvements for the bindings. This would benefit the
user-community, as well as give me more insight in what people would like to
see added to Motif++. Motif++ is still very much a voluntary project, and this
way I can make a list of priorities, in what order things should be added, or
changed.

If you're interested in joining such a mailing-list, please take the time to
reply to this message, and tell me so. When there is sufficient interest, say
about 20 people or more, 

En este caso se observa que todos los textos tratan sobre una biblioteca para la creación de entornos gráficos llamada Motif. No obstante ello, la temática abordada respecto a esa librería es distinta en cada uno de los textos.

In [67]:
# Documento 3
index_doc = 2

print("DOCUMENTO: ", index_doc+1, "  - TIPO: ", newsgroups_train.target_names[Ys_train[index_doc]])
print(newsgroups_train.data[idxs[index_doc]])
print("\n----------------------------------------------------------------\n")
for i in range(5):
    print(newsgroups_train.data[result_idxs[index_doc,i]])
    print("\n\n/////////////////////////////\n\n")

DOCUMENTO:  3   - TIPO:  soc.religion.christian


      Why should he have been any different "then"? Ozzy Osbourne,
ex-singer and main character of the Black Sabbath of good ole days past,
is and always was a devout catholic. Or so I've heard over on the
alt.rock-n-roll.metal newsgroups, an' I figure those folks oughta know..


----------------------------------------------------------------

: I may be wrong, but wasn't Jeff Fenholt part of Black Sabbath?  He's a
: MAJOR brother in Christ now.  He totally changed his life around, and
: he and his wife go on tours singing, witnessing, and spreading the
: gospel for Christ.  I may be wrong about Black Sabbath, but I know he
: was in a similar band if it wasn't that particular group...

   Yes, but Jeff also speaks out against listening to bands like Black
Sabbath. He says they're into all sorts of satanic stuff. I don't know.


/////////////////////////////


I have been following this thread on talk.religion,
soc.religion.christian.bi

En este caso observamos que la palabra Sabbath se repite en todos los documentos. No en todos se utiliza con el mismo significado, en algunos hace referencia a la banda de rock Black Sabbath y en otros en el día sagrado de la religión judía. 

In [68]:
# Documento 4
index_doc = 3

print("DOCUMENTO: ", index_doc+1, "  - TIPO: ", newsgroups_train.target_names[Ys_train[index_doc]])
print(newsgroups_train.data[idxs[index_doc]])
print("\n----------------------------------------------------------------\n")
for i in range(5):
    print(newsgroups_train.data[result_idxs[index_doc,i]])
    print("\n\n/////////////////////////////\n\n")

DOCUMENTO:  4   - TIPO:  sci.space

: >There is an emergency oxygen system that is capable of maintaining a
: >breathable atmosphere in the cabin for long enough to come down, even
: >if there is something like a 5cm hole in the wall that nobody tries
: >to plug.

Josh Hopkins (jbh55289@uxa.cso.uiuc.edu) replied:
: Wow.

: Double wow.  Can you land a shuttle with a 5cm hole in the wall?

Personnally, I don't know, but I'd like to try it sometime.

Programmatically, yes, we can land an Orbiter with a 5 cm hole in
the wall -- provided that the thing which caused 5 cm hole didn't
cause a Crit 1 failure on some of the internal systems.  There are
a few places where a 5 cm hole would cause a Bad Day -- especially
if the 5 cm hole went all the way through the Orbiter and out the
other side, as could easily happen with a meteor strike.  But a
hole in the pressure vessel would cause us to immediately de-orbit
to the next available landing site.

-- Ken Jenks, NASA/JSC/GM2, Space Shuttle Progra

En todos los casos se observa la dirección de correo electrónico kjenks@gothamcity.jsc.nasa.gov. Si bien todos los documentos tienen la misma categoría, la temática de cada uno de los documentos es distinta.

In [69]:
# Documento 5
index_doc = 4

print("DOCUMENTO: ", index_doc+1, "  - TIPO: ", newsgroups_train.target_names[Ys_train[index_doc]])
print(newsgroups_train.data[idxs[index_doc]])
print("\n----------------------------------------------------------------\n")
for i in range(5):
    print(newsgroups_train.data[result_idxs[index_doc,i]])
    print("\n\n/////////////////////////////\n\n")

DOCUMENTO:  5   - TIPO:  talk.religion.misc

Only my common sense.  The fire was caused by either Koresh and his
followers or by the FBI/ATF/CIA/KGB/and maybe the Harper Valley PTA.  Since
you are throwing around the evidence arguement, I'll throw it back.  Can
you prove any government agency did it?  (Please don't resort to "they 
covered it up so that proves they did it" or any wild theories about how
the government agencies intentionally started the fire.  The key words
are proof and evidence.)
proves they did it"


Please explain how Koresh was defending himself from those children who
burned.  


----------------------------------------------------------------



How do you know? Were you there?

While obviously Koresh was a nut case, the (typical) inability of the
government/media to get its story straight is quite disturbing. On
tuesday night, NBC news reported that the FBI did not know the place
was burning down until they saw black smoke billowing from the
building. The next d

Se observa que en todos los documentos tratan sobre Koresh (lider sectario). Algunas palabras que se repiten, son FBI y BATF. Si bien las categorías de los documentos no son las mismas podemos concluir que los documentos tratan sobre la misma temática.

In [445]:
print(result_idxs[3,0])
print(newsgroups_train.data[9268])

9268

But you still need the pitching staff to hold the opposing team to
one run.


**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.




Teniendo en cuenta que los textos están en Inglés, iniciamos el vectorizador utilizando el parametro stop_words='english'. De esta forma las palabras que estan en la lista no son tomadas en cuenta al momento de generar el vocabulario para vectorizar. 
A su vez, el atributo smooth_df se setea False de forma de eliminar ese documento (el cual tiene todo el vocabulario) de más que la libreria agrega a los efectos de evitar la división por cero.

In [390]:
tfidfvect2 = TfidfVectorizer(stop_words='english', smooth_idf=True)#, max_df=0.1)
X2_train = tfidfvect2.fit_transform(newsgroups_train.data)
y2_train = newsgroups_train.target
print(X_train.shape)
print(X2_train.shape)
print(y2_train.shape)

(11314, 101631)
(11314, 101322)
(11314,)


In [391]:
nb_cl = MultinomialNB(alpha=.01)
nb_cl.fit(X2_train, y2_train)
cm_cl = ComplementNB(alpha=.18)
cm_cl.fit(X2_train, y2_train)

ComplementNB(alpha=0.18)

In [392]:
X2_test = tfidfvect2.transform(newsgroups_test.data)
y2_test = newsgroups_test.target
print(X2_test.shape)
print(y2_test.shape)
y2_pred_nb = nb_cl.predict(X2_test)
y2_pred_cm = cm_cl.predict(X2_test)

(7532, 101322)
(7532,)


In [393]:
print("F1-score NB: ", f1_score(y2_test, y2_pred_nb, average='macro'))
print("F1-score CM: ", f1_score(y2_test, y2_pred_cm, average='macro'))

F1-score NB:  0.6844389919212164
F1-score CM:  0.6972162188083315



**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [429]:
idx2word_2 = {v: k for k, v in tfidfvect2.vocabulary_.items()}
X2_train_T =X2_train.T
# SELECCIONAMOS LA PALABRA 'car', 'hello', 'people', 'flag' y 'children' 
word_list = ['doctor', 'tree', 'mail', 'egg', 'money']
words_idx = [tfidfvect2.vocabulary_[word] for word in word_list]

X2s_train_T = X2_train_T[words_idx]

word_cossim = cosine_similarity(X2s_train_T, X2_train_T)

for i, element in enumerate(word_cossim):
    print("\n\n Word: ", word_list[i], "IDX: ", words_idx[i])
    result_idxs[i] = np.argsort(element)[::-1][1:6]
    
    target_list = list()
    print("Target primeros 5 documentos con mas similitud:")
    for sort_idx in result_idxs[i]:
        target_list.append(idx2word_2[sort_idx])
    print(target_list)
    print("IDX primeras 5 palabras con mas similitud:")
    print(result_idxs[i])
    
    



 Word:  doctor IDX:  34571
Target primeros 5 documentos con mas similitud:
['receptionist', 'clinic', 'misbehavior', 'angering', 'urgent']
IDX primeras 5 palabras con mas similitud:
[76401 27839 61837 18577 92530]


 Word:  tree IDX:  89875
Target primeros 5 documentos con mas similitud:
['decorations', 'flourish', 'immaculate', 'withering', 'attest']
IDX primeras 5 palabras con mas similitud:
[32279 40741 49035 96633 20269]


 Word:  mail IDX:  59071
Target primeros 5 documentos con mas similitud:
['address', 'thanks', 'send', 'immensely', 'regualar']
IDX primeras 5 palabras con mas similitud:
[16818 88277 81378 49052 76859]


 Word:  egg IDX:  36376
Target primeros 5 documentos con mas similitud:
['chicken', 'imperialism', 'blasphemous', 'begs', 'xmncolormap']
IDX primeras 5 palabras con mas similitud:
[26937 49163 23083 22140 98389]


 Word:  money IDX:  62794
Target primeros 5 documentos con mas similitud:
['buyback', 'computerize', 'spend', 'fundraising', 'fund']
IDX primeras 5 

Según los resultados obtenidos, vemos que en muchos casos algunas de las 5 palabras con más similitud tienen relación con la palabra seleccionada.
 
Por ejemplo:
- La palabra doctor nos devuelve receptionist, clinic, urgent
- La palabra tree nos devuelve decorations, flourish, withering
- La palabra mail nos devuelve address, thanks, send
- La palabra egg nos devuelve chicken  
- La palabra money nos devuelve buyback, spend, fundraising, fund